<a href="https://colab.research.google.com/github/FinalYearProject-Siddha/Code/blob/main/Second_Review.All-Ouputs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ACTIVATING GPU

In [ ]:
!nvidia-smi

In [2]:
import torch
torch.cuda.is_available()

import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

### INSTALLING REQUIRED PACKAGES

In [ ]:
!pip install stanza

In [ ]:
!sudo apt-get install graphviz graphviz-dev # to plot graphs
!pip install pygraphviz 

In [5]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
import stanza
import networkx as nx
import matplotlib.pyplot as plt
from pygraphviz import *
import pandas as pd
import warnings
warnings.filterwarnings("ignore") 

CONFIGURING THE OUTPUT WINDOW

In [6]:
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

### PROCESSING DATA

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
with open('/content/drive/MyDrive/Final Year Project/Outputs/Preprocessed.txt','r') as f:
  text = f.read()

In [9]:
Sentences = text.split("\n")
Sentences

['அக்ரகாரம்',
 'அக்ரகாரத்தைப் 100 கிராம் பொடி செய்து கிராம் பொடியைத் தேனில் கலந்து சாப்பிட்டுவந்தால் திக்குவாய் குணமாகும்',
 'அக்ரகாரம் திப்பிலி  இரண்டையும் சம அளவு எடுத்து அரைத்துப் பொடி செய்து இரண்டு கிராம் பொடியைத் தேனில் குழைத்துச் சாப்பிட்டு வந்தால் நுரையீரல் சார்ந்த நோய்கள் அனைத்தும் தீரும்',
 'அக்ரகாரத்தை வல்லாரைச் சாறு சேர்த்து அரைத்து தினமும் பட்டாணி அளவு சாப்பிட்டு வந்தால் காக்காய் வலிப்பு நோய் குணமாகும்',
 'அக்ரகாரத்தைப் பொடி செய்து இரண்டு கிராம் பொடியை நாட்டுக்கோழி முட்டையின் மஞ்சள் கருவில் குழைத்துச் சாப்பிட்டால் உடலுறவு வேட்கை அதிகரிக்கும்',
 'கொட்டைப்பாக்கு ஊறவைத்த தண்ணீரில் அக்ரகாரத்தைச் சேர்த்து அரைத்து தினமும் சுண்டைக்காய் அளவு சாப்பிட்டால் உடலுறவு வேட்கை அதிகரிக்கும் உடலுறவு நேரமும் நீடிக்கும்',
 'அக்ரகாரம் அதிமதுரம் கரிசலாங்கண்ணி  மூன்றையும் சம அளவு எடுத்து அரைத்து தினமும் இரண்டு கிராம் அளவு அதிகாலையில் சாப்பிட்டு வந்தால் குரல் இனிமையாகும்',
 'அக்ரகாரம் ஆடாதொடச நிலவேம்பு  மூன்றையும் சம அளவு எடுத்துப் பொடி செய்து இரண்டு கிராம் பொடியை மூன்று வேளையும் தேனில் குழைத்துச்

### STANZA NEURAL PIPELINE

CONFIGURING STANZA AND NETWORKX

In [10]:
LANG = "ta"
NXDOPTS = {
    "edge_color": "black",
    "node_size": 80,
    "width": 1,
}

In [11]:
# download appropriate language model
stanza.download(LANG, verbose=False)
nlp = stanza.Pipeline(
    LANG,
    processors="tokenize, pos, lemma, mwt, depparse",
    verbose=False,
    use_gpu=False,
)

INITIALISATION

In [12]:
diseases = set()
ingredients = set()
plants = set()
rels = {}
byProducts = {}
similarIngredients = {}
similarDiseases = {}

In [13]:
REJECTED_WORDS = ["போன்றவை", "நோய்களும்", "அனைத்தும்", "போன்றம்", "தினம்", "கழுவினால்", 'தோய்த்துக்கொண்டால்', 'பழத்தை', 'தீரும்', 'குண']
REJECTED_DISEASES = ["நோய்", 'பாதிப்பு', 'வடிதல்', '€த ரத்த', 'குப்பைக்', 'பச', 'வடு', 'நிவாரணம்', 'ஒத்தடம்', 'புதினாவை', 'வதக்கிச்', 'வளர்ச்சி', 'வளம்', 'கோளாறுகள்', 'நிறம்', 'பொலிவு', 'ஆர்வம்', 'முடி', 'கடுப்பு', 'வளமை', 'இன்பம்', 'உலை', 'குணம்', 'பழுத்து', 'பூ பாதிப்பு', 'வெற்றிலையைத்', 'கீரை', 'சதை', 'கீரைய்', 'தடவினால']
PLANT_ASSOCIATE = ['பருப்பு', 'விதை', 'எண்ணெய்', 'கீரை', 'பட்டை', 'இலை', 'மரப்', 'மரப்பட்டை', 'பழம்', 'பழத்', 'பூ', 'பூவை', 'வேர்பு', 'கொடி', 'தோலை', 'பூவு', 'கனி', 'வற்றல்', 'பூக்கள்', 'பழ', 'மாவை']
REJECTED_PLANTS = ['லிட்டர்', 'கிராம்', 'உடல்', 'அளவு', 'டம்ளர்', 'உலர்ந்த', 'அவர்', 'வேகவை', 'துருவு', 'அனைத்து', 'ஸ்பூன்', 'பழங்கு', 'றினால்', 'சாப்பிட்டுவந்தால்', 'வன்னி', 'வீக்கம்', 'நோய்', 'கூந்தல்', 'காயம்', 'ஆகவ', 'நாள்', 'இரவு', 'அது', 'தலா50', 'சாப்பிட்டுவம்', 'பவுடராக்கி', 'மாவாக்கி', 'சமஅளவு', 'ச', 'மணி', 'ரத்த', 'ஒருநாள்', 'தளர்ச்சி', 'சதை', 'வேகவைத்து', 'சாப்பிட்', 'காய்ச்சி', 'குடல்', 'வலி', 'பிடிப்பு', 'காய்ச்சிக்', 'காலை', 'வாய்', 'வால்', 'கஷாயமாக்கி', 'கடல்', 'உள்ளவர்கள் குணம்']
PRODUCTS = ['சாறு', 'சாற்று', 'பொடி', 'கஷாயம்', 'ஜூஸ்', 'விழுதை', 'சூப்பாக', 'தூள்', 'தைலம்']
ACCEPTED_PLANT_RELATIONS = ['nmod', 'nsubj', 'obj']
OTHER_PLANT_RELATIONS = ['nummod', 'advmod', 'advcl', 'amod']

### BASE FUNCTIONS

In [14]:
def init_values():
  constants = {}
  constants["word_rels"] = {} # word relationships (deprels) 
  constants["word_maps"] = [] # word
  constants["lemma_maps"] = [] # lemma
  constants["df"] = pd.DataFrame() # to display all values
  constants["successors"] = {} # to store successors of all nodes
  constants["leaf_nodes"] = [] # to store leaf nodes
  constants["root"] = '' # to store root node
  constants["diseases"] = set() # to store diseases of each line
  constants["ingredients"] = set() # to store ingredients of each line
  constants["plants"] = set() # to store plants of each line
  constants["G"] = nx.DiGraph() # create an instance of a graph
  return constants

In [15]:
def create_graph(sentence, constants):
  # fictious root node
  constants["G"].add_node(0, id=0, text="ROOT", lemma="ROOT", upos="ROOT")  
  # iterate through all words in a sentence
  for word in [w for w in sentence.words]:
    wordict = word.to_dict()
    # add all word_id's as nodes to graph
    constants["G"].add_node(wordict["id"], **wordict)
    # connect edges between words and their head elements with label as the deprel
    constants["G"].add_edge(wordict["head"], wordict["id"], label=wordict["deprel"], length=100)
    # store relationships of the word
    constants["word_rels"][wordict['id']] = wordict['deprel']
    # store words 
    constants["word_maps"].append(wordict['text'])
    # store lemma
    constants["lemma_maps"].append(wordict['lemma'])
    # store root element based on deprel
    if wordict["deprel"]=="root":
      constants["root"] = wordict["id"]
    # store successors of each node
    constants["successors"][wordict["id"]] = list(constants["G"].successors(wordict["id"]))
    # store the word dictionary in a dataframe
    constants["df"] = constants["df"].append(wordict, ignore_index=True)
  return constants

In [16]:
def display_graph(G):
  # fix layout
  pos = nx.drawing.nx_agraph.graphviz_layout(G, prog='dot')
  pos = nx.nx_agraph.graphviz_layout(G)
  # configure graph settings
  nodelabels = nx.get_node_attributes(G, "id")
  edgelabels = nx.get_edge_attributes(G, "label")
  # display the graphs
  nx.draw(G, pos, with_labels=True, labels=nodelabels, **NXDOPTS)
  nx.draw_networkx_edge_labels(G, pos, edge_labels=edgelabels, font_size=8, verticalalignment="top")
  plt.figure(1, figsize=(200, 80), dpi=200)
  plt.show()
  # clear the graph to repeat for another sentence
  G.clear()

In [17]:
def add_leaf_nodes(successors):
  leaf_nodes = []
  # check every node in successors list and add to leaf nodes if no successors are found
  for successor in successors.keys():
    if successors[successor] == []:
      leaf_nodes.append(successor)
  return leaf_nodes

In [18]:
# to check for relationships in the same level of the graph
def check_relationship_in_level(constants, relation):
  # to store the word with given relationship 
  rel_check = ''
  for node in constants["successors"][constants["root"]]:
    # to check for relation, validate the word and validate the distance between root word
    if(constants["word_rels"][node] == relation and constants["word_maps"][node-1] not in REJECTED_WORDS and constants["root"]-node<=5):
      rel_check = constants["word_maps"][node-1]
  return rel_check

In [19]:
def display_constants(constants):
  sentence = " ".join([word for word in constants["word_maps"]])
  print("Sentence: ", sentence)
  display(constants["df"])
  print("Word-Relationships: ", constants["word_rels"])
  print("Word-Mappings: ", constants["word_maps"])
  print("Lemma-Mappings: ", constants["lemma_maps"])
  print("Root-Node: ", constants["root"])
  print("Successors: ", constants["successors"])
  print("Leaf-Nodes: ", constants["leaf_nodes"])
  display_graph(constants["G"])
  print("\n\n\n")

In [20]:
def LCS(X,Y):
  m = len(X)
  n = len(Y)
  LCSuff = [[0 for k in range(n+1)] for l in range(m+1)]
 
    # To store the length of
    # longest common substring
  result = 0
 
    # Following steps to build
    # LCSuff[m+1][n+1] in bottom up fashion
  for i in range(m + 1):
      for j in range(n + 1):
          if (i == 0 or j == 0):
              LCSuff[i][j] = 0
          elif (X[i-1] == Y[j-1]):
              LCSuff[i][j] = LCSuff[i-1][j-1] + 1
              result = max(result, LCSuff[i][j])
          else:
              LCSuff[i][j] = 0
  return result

In [34]:
def get_synsets(word_list, similarNames):
  rejectList = ['கீரை', 'மரம்', 'மரப்பட்டை', 'பருப்பு', 'பூ']
  for i in range(len(word_list)):
    for j in range(0, len(word_list)):
      if(i != j):
        first_word = " ".join([w for w in word_list[i].split() if w not in rejectList])
        second_word = " ".join([w for w in word_list[j].split() if w not in rejectList])
        if(max(len(first_word),len(second_word))>0):
          sim = LCS(first_word, second_word)/max(len(first_word),len(second_word))
          if sim>=0.60:
            if word_list[i] not in similarNames.keys():
                similarNames[word_list[i]] = {word_list[j]}
            else:
                similarNames[word_list[i]].add(word_list[j])
  return similarNames
  #       for word in rejectList:
  #         if word in word_list[j] and word in word_list[i]:
  #           flag = 1
  #       if flag!=1:
  #         sim = LCS(word_list[i],word_list[j])/max(len(word_list[i]),len(word_list[j]))
  #         if sim >= 0.60: #change to 0.7
  #           if word_list[i] not in similarNames.keys():
  #             similarNames[word_list[i]] = {word_list[j]}
  #           else:
  #             similarNames[word_list[i]].add(word_list[j])
  # return similarNames

### RULES FOR CATEGORY TAGGING

RULES FOR MEDICINAL PLANTS

In [22]:
def get_plants(constants):
  plants = set()
  if(len(constants["word_maps"])<3):
    plants.add(" ".join([word for word in constants["word_maps"]]))
  return plants

RULES FOR DISEASES

CORE RULES<br><br>
Case 1: If `obj` relationship is a successor of root, a valid word and in valid distance from root node. <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Case 1.1: If `obj` relationship is a leaf node, add word[`obj`].<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Case 1.2: If `nmod` relationship exists in successors of `obj` node, the word is valid and at a valid distance, add "word[`nmod`] + word[`obj`]".<br><br>
Case 2: If `nsubj` relationship is a successor of root, a valid word and in valid distance from root node. <br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Case 2.1: If `nmod` relationship exists in successors of `obj` node, the word is valid and at a valid distance, add "word[`nmod`] + word[`nsubj`]".<br>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Case 2.2: If `obj` relationship exists in the same successor level, the word is valid and at a valid distance, add "word[`nsubj`] + word[`obj`]"<br>

In [23]:
def get_diseases(constants):
        diseases = set()
        for node in constants["successors"][constants["root"]]:
          # case 1: obj successor of root
          if(constants["word_rels"][node] == 'obj' and constants["word_maps"][node-1] not in REJECTED_WORDS and constants["root"]-node<=5):
            # case 1.1: root->obj(leaf)
            if node in constants["leaf_nodes"] and constants["word_maps"][node-1] not in REJECTED_WORDS and constants["root"]-node<=5 and constants["word_maps"][node-1] not in REJECTED_DISEASES:
              diseases.add(constants["word_maps"][node-1])
            # case 1.2: root->obj->nmod 
            for sub_node in constants["successors"][node]:
              if(constants["word_rels"][sub_node] == 'nmod' and constants["word_maps"][sub_node-1] not in REJECTED_WORDS and constants["root"]-sub_node<=5 and constants["word_maps"][sub_node-1]+" "+constants["word_maps"][node-1] not in REJECTED_DISEASES):
                diseases.add(constants["word_maps"][sub_node-1]+" "+constants["word_maps"][node-1])
          # case 2: nsubj successor of root
          elif(constants["word_rels"][node] == 'nsubj' and constants["word_maps"][node-1] not in REJECTED_WORDS):
            # case 2.1: root->nsubj->nmod
            for sub_node in constants["successors"][node]:
              if(constants["word_rels"][sub_node] == 'nmod' and constants["word_maps"][sub_node-1] not in REJECTED_WORDS  and constants["root"]-sub_node<=5 and constants["word_maps"][sub_node-1]+ " "+ constants["word_maps"][node-1] not in REJECTED_DISEASES):
                diseases.add(constants["word_maps"][sub_node-1]+ " "+ constants["word_maps"][node-1])
            # case 2.2: root->(nsubj+obj)
            obj_check = check_relationship_in_level(constants, 'obj')
            if obj_check != '' and constants["word_maps"][node-1]+ " "+ obj_check not in REJECTED_DISEASES:
              diseases.add(constants["word_maps"][node-1]+ " "+ obj_check)
        return diseases

RULES FOR CORE INGREDIENTS

CORE RULES <br>
Case 1: If the `node` is in accepted relation list and the successor nodes of every `node` are in the accepted relation list and leaf nodes, then add the `lemma of the successor nodes` and the `node`. <br>
Case 2. If the `node` is not in accepted relation list and their successor nodes are leaf nodes and in the acceptable relation list, then add the `lemma of successor nodes`.

In [24]:
def get_ingredients(constants, plants):
  ingredients = set()
  for node in constants["successors"].keys():
    # case 1
    # check if the node is in ACCEPTED_PLANT_RELATIONS
    if constants["word_rels"][node] in ACCEPTED_PLANT_RELATIONS:
      # check if successors of node is a subset of leaf nodes and not empty
      if(set(constants["successors"][node]).issubset(set(constants["leaf_nodes"])) and constants["successors"][node] != []):
        # get all the plants in the sentence
        plant_string = " ".join([constants["lemma_maps"][n-1] for n in constants["successors"][node] if constants["word_rels"][n] in ACCEPTED_PLANT_RELATIONS])
        temp_plants = plant_string.split()
        # to join 2 or more phrases eg: identifying 'அகத்தி கீரை' from ['அகத்தி', 'கீரை', ....]
        i=0
        while(i<=len(temp_plants)-1):
          # check for associate words and add the word before together as a plant, remove the words added
          if temp_plants[i] in PLANT_ASSOCIATE and len(temp_plants)!=0:
            t1 = temp_plants[i-1]
            t2 = temp_plants[i]
            if(t1+" "+t2 not in plants and t1+" "+t2 not in REJECTED_PLANTS+PLANT_ASSOCIATE+PRODUCTS):
              ingredients.add(t1+" "+t2)
              del temp_plants[i-1:i+1]
              i=0
            if(t1+" "+t2 in plants):
              constants["plants"].add(t1+" "+t2)
          i=i+1
        # adding the remaining words in the string as single ingredients (no associates)
        ingredients.update(set([tp for tp in temp_plants if tp not in plants and tp not in REJECTED_PLANTS+PLANT_ASSOCIATE+PRODUCTS]))
        for tp in temp_plants:
          if tp in plants:
            constants["plants"].add(tp)
        # add the node also since it satisfied the acceptable relation
        if constants["lemma_maps"][node-1] not in plants and constants["lemma_maps"][node-1] not in REJECTED_PLANTS+PLANT_ASSOCIATE+PRODUCTS:
          ingredients.add(constants["lemma_maps"][node-1])
        elif constants["lemma_maps"][node-1] in plants:
          constants["plants"].add(constants["lemma_maps"][node-1])
    # case 2
    elif constants["word_rels"][node] not in ACCEPTED_PLANT_RELATIONS and constants["successors"][node] != []:
      # if the successor nodes are all leaf nodes, add all the nodes separately.
      if(set(constants["successors"][node]).issubset(set(constants["leaf_nodes"]))):
        ingredients.update([constants["lemma_maps"][n-1] for n in constants["successors"][node] if constants["word_rels"][n] in ACCEPTED_PLANT_RELATIONS and constants["lemma_maps"][n-1] not in plants and constants["lemma_maps"][n-1] not in REJECTED_PLANTS+PLANT_ASSOCIATE+PRODUCTS])
        for n in constants["successors"][node]:
          if constants["lemma_maps"][n] in plants:
            constants["plants"].add(constants["lemma_maps"][n])
      else:
        break  
  return ingredients
        

BY PRODUCTS

In [25]:
def get_by_products(constants):
  for i in range(len(constants["lemma_maps"])):
    if constants["lemma_maps"][i] in PRODUCTS:
      idx = i-1
      for j in range(idx,0,-1):
        if(constants["lemma_maps"][j] in ingredients and constants["lemma_maps"][j] not in set(REJECTED_PLANTS+REJECTED_WORDS+PLANT_ASSOCIATE+PRODUCTS)):
          if constants["lemma_maps"][i] not in byProducts.keys():
            byProducts[constants["lemma_maps"][i]] = {constants["lemma_maps"][j]}
          else:
            byProducts[constants["lemma_maps"][i]].add(constants["lemma_maps"][j])
          break

RELATIONSHIPS

In [26]:
def get_relations(constants):
  # defining all the relationships for a plant: supp-ing, dis, byprod, sim-prod
  # defining all the relationships for a disease: sim-dis, med-plant, ing
  for disease in constants["diseases"]:
    for plant in constants["plants"]:
      if disease not in rels.keys():
        rels[disease] = {(plant, 'med-plant')}  
      else:
        rels[disease].add((plant, 'med-plant'))
      if plant not in rels.keys(): 
        rels[plant] = {(disease, 'dis')}
      else:
        rels[plant].add((disease, 'dis'))
  
  for disease in constants["diseases"]:
    for ingredient in constants["ingredients"]:
      if disease not in rels.keys():
        rels[disease] = {(ingredient, 'ing')}  
      else:
        rels[disease].add((ingredient, 'ing'))
      if ingredient not in rels.keys(): 
        rels[ingredient] = {(disease, 'dis')}
      else:
        rels[ingredient].add((disease, 'dis'))

  for plant in constants["plants"]:
    for ingredient in constants["ingredients"]:
      if plant not in rels.keys():
        rels[plant] = {(ingredient, 'supp-ing')}  
      else:
        rels[plant].add((ingredient, 'supp-ing'))
      if ingredient not in rels.keys():
        rels[ingredient] = {(plant, 'supp-ing')}  
      else:
        rels[ingredient].add((plant, 'supp-ing'))

  for byprod in byProducts.keys():
    for plants in byProducts[byprod]:
      if byprod not in rels.keys():
        rels[byprod] = {(plants, 'byprod')}
      else:
        rels[byprod].add((plants, 'byprod'))
  
  for simDis in similarDiseases.keys():
    for dis in similarDiseases[simDis]:
      if simDis not in rels.keys():
        rels[simDis] = {(dis, 'sim-dis')}
      else:
        rels[simDis].add((dis, 'sim-dis'))
      if dis not in rels.keys():
        rels[dis] = {(simDis, 'sim-dis')}
      else:
        rels[dis].add((simDis, 'sim-dis'))
  
  for simPlant in similarIngredients.keys():
    for plant in similarIngredients[simPlant]:
      if simPlant not in rels.keys():
        rels[simPlant] = {(plant, 'sim-plant')}
      else:
        rels[simPlant].add((plant, 'sim-plant'))
      if plant not in rels.keys():
        rels[plant] = {(simPlant, 'sim-plant')}
      else:
        rels[plant].add((simPlant, 'sim-plant'))

In [27]:
def get_semantically_similar_plants(rels, plant):
  sim_plants = set()
  diseases_cured = set([k for (k,v) in rels[plant] if v == "dis"])
  for pl in ingredients.union(plants):
    if pl in rels.keys() and pl != plant:
      dis_cured = set([k for (k,v) in rels[pl] if v == 'dis'])
      if(diseases_cured <= dis_cured and len(dis_cured)>0 and len(diseases_cured)>0):
        sim_plants.add(pl)
  return sim_plants

### APPLYING CATEOGORY TAGGING TO EACH SENTENCE

In [ ]:
for sentences in Sentences: 
    doc = nlp(sentences) # fit the sentence in the neural pipeline
    for sentence in doc.sentences: 
      # init values
      constants = init_values()
      # create graph and set values
      constants = create_graph(sentence, constants)
      # get leaf nodes
      constants["leaf_nodes"] = add_leaf_nodes(constants["successors"])
      #get all plants
      plants.update(get_plants(constants))
      # category tagging for ingredients in the sentence
      ingredients.update(get_ingredients(constants, plants))
      # category tagging for plants in the sentence
      get_plants(constants)
      # category tagging for diseases in the sentence
      diseases.update(get_diseases(constants))
      # category tagging for byproducts in the sentence
      get_by_products(constants)
      # display
      display_constants(constants)

### CATEGORY TAGGING RESULTS

CORE INGREDIENTS

In [29]:
ingredients_df = pd.DataFrame(list(ingredients))
ingredients_df

,0
0,வெந்தயக்
1,விழுது
2,துயிலி
3,அசோகப்பட்டை
4,மாதுளம் வேர்பு
5,படை
6,நெல்லி
7,கல்யாணமுருங்கை
8,தேற்றான்கொட்டை
9,வசீகரம்


DISEASES

In [30]:
diseases_df = pd.DataFrame(list(diseases))
diseases_df

,0
0,நாள்பட்ட மலச்சிக்கல்
1,நமைச்சல்
2,கூச்சம் ஆடுதல்
3,கர்ப்பப் கோளாறுகள்
4,கண் அரிப்பு
5,தாது விருத்தி
6,ஆன எடை
7,வேதனை வலி
8,உயிரணுக்களின் எண்ணிக்கை
9,எடை நோய்களைக்


PLANTS

In [31]:
plants_df = pd.DataFrame(list(plants))
plants_df

,0
0,இசங்கு
1,உத்தாமணி
2,அதிமதுரம்
3,இருவாட்சி
4,சர்ப்பகந்தா
5,பசலைக் கீரை
6,பருப்புக் கீரை
7,இலந்தை
8,கடுகு
9,இத்தி


In [32]:
byProducts

{'பொடி': {'அதிமதுரம்',
  'அரை',
  'ஆகியவற்றைப்',
  'இப்பிலி',
  'இரண்டு',
  'இரண்டையும்',
  'இராம்',
  'இலவம்',
  'உப்பு',
  'ஊறவை',
  'எருக்கம்',
  'எலுமிச்சம்',
  'ஏலக்கா',
  'ஒன்று',
  'கசகசா',
  'கடுக்காய்',
  'கட்டை',
  'கம்',
  'கராம்',
  'கழற்சிக்காய்',
  'காயவை',
  'கால்',
  'கிராம்பு',
  'கிழங்கு',
  'கிழங்கை',
  'கீரையுடன்',
  'கீரையை',
  'கீழாநெல்லி',
  'குச்சி',
  'கொட்டை',
  'சடை',
  'சர்க்கீரை',
  'சிட்டிகை',
  'சீரகம்',
  'சுக்கு',
  'செய்துகொள்ளவ்',
  'ஜாதிக்காய்',
  'தனமும்',
  'தனியா',
  'தப்பிலி',
  'தான்றிக்காய்',
  'திப்பிலி',
  'துத்தி',
  'தும்மட்டி',
  'நாவல்',
  'நெய்',
  'படிகாரம்',
  'பால்',
  'பூக்களை',
  'பெருங்காயம்',
  'பொடியாக்கி',
  'மஞ்சள்',
  'மர',
  'மரம்',
  'மரம்து',
  'மிளகு',
  'மூன்றை',
  'மொட்டு',
  'லவங்கம்',
  'வற்றலை',
  'வாய்விளங்கம்',
  'வாழைக்கா',
  'வேரை',
  'வேர்'},
 'சாறு': {'அகத்தி',
  'அருகம்பு',
  'அரை',
  'ஆரஞ்சு',
  'இஞ்சி',
  'இரண்டு',
  'இரண்டையும்',
  'உப்பு',
  'எருக்கம்',
  'எலுமிச்சம்',
  'எலுமிச்சை',
  'கரிசலாங்கண்ணி',
  'க

### SYNSET IDENTIFICATION

SIMILAR PLANTS 

In [35]:
print(get_synsets(list(ingredients)+list(plants), similarIngredients))

{'வெந்தயக்': {'வெந்தயக்கீரை', 'வெந்தயக் கீரை', 'வெந்தயத்தை', 'வெந்தயம்'}, 'விழுது': {'விழு'}, 'துயிலி': {'துயிலி கீரை', 'துயிலிக் கீரை'}, 'அசோகப்பட்டை': {'லவங்கப்பட்டை'}, 'படை': {'சடை'}, 'நெல்லி': {'மில்லி', 'அல்லி', 'அருநெல்லி', 'நெல்லிக்', 'நெல்லிக்கா', 'கீழாநெல்லி', 'நெல்', 'நெல்லிம்'}, 'கல்யாணமுருங்கை': {'கல்யாணமுருங்கைக் கீரை', 'கல்யாணமுருங்கை கீரை'}, 'மணலி': {'மணலி கீரை', 'மணலிக் கீரை'}, 'தினை': {'தினம்'}, 'ஏலரிசி': {'அரிசி'}, 'ஆவாரம் இலை': {'ஆவாரம்', 'ஆமணம் இலை', 'ஆவாரம் பூவை', 'ஆவாரம் பூ', 'ஆவாரம்பூ', 'இசம் இலை'}, 'அத்திப்': {'அத்தி'}, 'மஞ்சள் கீரை': {'மஞ்சள்', 'மஞ்சள்தூளை'}, 'அரை': {'அரைக் கீரை', 'ஆரை கீரை', 'ரை', 'அரை கீரை', 'அரச மரம்', 'ஆரை'}, 'பூண்டு கீரை': {'தண்டு', 'தண்டு கீரை', 'பூண்டை', 'துண்டு', 'இரண்டு', 'பூண்டு'}, 'கிழங்கு': {'கிழங்கை'}, 'சளி': {'ஆளி'}, 'முடக்கம் கீரை': {'முடக்கம்', 'எருக்கம்'}, 'பாதாம்': {'பாதாம் பருப்பு'}, 'மருதம் மரப்பட்டை': {'மருதம்'}, 'கொடிப்பசலை': {'கொடிப்பசலை கீரை', 'கொடிப்பசலைக் கீரை'}, 'சுக்காங்': {'சுக்காங் கீரை', 'கடுக்கா', 'சுக்குக் கீரை'

SIMILAR DISEASES

In [36]:
print(get_synsets(list(diseases), similarDiseases))

{'நாள்பட்ட மலச்சிக்கல்': {'டால் மலச்சிக்கல்', 'மில்லி மலச்சிக்கல்'}, 'நமைச்சல்': {'எரிச்சல்', 'கழிச்சல்', 'படை நமைச்சல்'}, 'கர்ப்பப் கோளாறுகள்': {'ஜீரணக் கோளாறுகள்', 'மனநலக் கோளாறுகள்', 'வாயுக் கோளாறுகள்', 'கணையக் கோளாறுகள்', 'சுவாசக் கோளாறுகள்', 'அஜீரணக் கோளாறுகள்', 'கல்லீரல் கோளாறுகள்', 'வாதக் கோளாறுகள்', 'குடல் கோளாறுகள்', 'பூச்சிக் கோளாறுகள்', 'நரம்புக் கோளாறுகள்'}, 'கண் அரிப்பு': {'வலி அரிப்பு', 'அரிப்பு', 'அரிப்பும்', 'உடல் அரிப்பு'}, 'தாது விருத்தி': {'ஆண்மை விருத்தி'}, 'எடை நோய்களைக்': {'தோல் நோய்களைக்', 'படை நோய்கள்', 'தொண்டை நோய்கள்', 'படை நோய்களைத்'}, 'வலி நோய்': {'கண் நோய்', 'இதய நோய்', 'காச நோய்', 'வாத நோய்', 'வலி நோய்கள்'}, 'அடிவயிற்று வலி': {'வயிற்று வலி'}, 'வீக்கம் குணம்': {'வீக்கம் வலி'}, 'தலைவலி வலி': {'தலைவலி', 'வலி வலி'}, 'உடல் அசதியும்': {'உடல் அசதி'}, 'மூளை சுறுசுறுப்பு': {'சுறுசுறுப்பு'}, 'பக்கவாதம் நோய்கள்': {'அழுத்தம் நோய்கள்', 'பித்தம் நோய்கள்'}, 'ரசம் கோளாறுகளை': {'ஜீரணக் கோளாறுகள்', 'மனநலக் கோளாறுகள்', 'வாயுக் கோளாறுகள்', 'பை கோளாறுகள்', 'சளி கோளாறுகள்', 'கண

### SAVING RESULTS

In [37]:
plants_df.to_csv('/content/drive/MyDrive/ingredients.csv')

In [38]:
diseases_df.to_csv('/content/drive/MyDrive/diseases.csv')

In [39]:
similarDiseases_df = pd.DataFrame.from_dict(similarDiseases, orient ='index') 
similarDiseases_df.to_csv('/content/drive/MyDrive/similarDiseases.csv') 

In [40]:
similarIngredients_df = pd.DataFrame.from_dict(similarIngredients, orient ='index') 
similarIngredients_df.to_csv('/content/drive/MyDrive/similarIngredients.csv') 

### RELATIONSHIP MAPPING


In [ ]:
for sentences in Sentences: 
    doc = nlp(sentences) # fit the sentence in the neural pipeline
    for sentence in doc.sentences: 
      # init values
      constants = init_values()
      # create graph and set values
      constants = create_graph(sentence, constants)
      # get leaf nodes
      constants["leaf_nodes"] = add_leaf_nodes(constants["successors"])
      # category tagging for plants in the sentence
      plants.update(get_plants(constants))
      # category tagging for
      constants["ingredients"] = get_ingredients(constants, plants)
      # category tagging for diseases in the sentence
      constants["diseases"] = get_diseases(constants)
      # set relationships
      constants["relationships"] = get_relations(constants)
      # category tagging for byproducts in the sentence
      get_by_products(constants)
      # display
      display_constants(constants)

In [42]:
for (key,value) in rels.items():
    print(key,":", value)

பொடி : {('இருவேளை', 'byprod'), ('கிராம்பு', 'byprod'), ('சடை', 'byprod'), ('கால்', 'byprod'), ('நெய்', 'byprod'), ('ஊறவை', 'byprod'), ('கீழாநெல்லி', 'byprod'), ('கீரையை', 'byprod'), ('கராம்', 'byprod'), ('செய்துகொள்ளவ்', 'byprod'), ('மரம்து', 'byprod'), ('பால்', 'byprod'), ('இலவம்', 'byprod'), ('சர்க்கீரை', 'byprod'), ('வேர்', 'byprod'), ('கிழங்கு', 'byprod'), ('வேரை', 'byprod'), ('பெருங்காயம்', 'byprod'), ('வற்று', 'byprod'), ('நாவல்', 'byprod'), ('எருக்கம்', 'byprod'), ('வற்றலை', 'byprod'), ('கொட்டை', 'byprod'), ('கசகசா', 'byprod'), ('மொட்டு', 'byprod'), ('தான்றிக்காய்', 'byprod'), ('இரண்டு', 'byprod'), ('துத்தி', 'byprod'), ('மர', 'byprod'), ('சிட்டிகை', 'byprod'), ('காயவை', 'byprod'), ('இரண்டையும்', 'byprod'), ('குச்சி', 'byprod'), ('ஏலக்கா', 'byprod'), ('சுக்கு', 'byprod'), ('தனமும்', 'byprod'), ('ஒன்று', 'byprod'), ('பூக்களை', 'byprod'), ('மூன்றை', 'byprod'), ('கழற்சிக்காய்', 'byprod'), ('பொடியாக்கி', 'byprod'), ('கீரையுடன்', 'byprod'), ('தும்மட்டி', 'byprod'), ('திப்பிலி', 'bypr

### RETRIEVAL

In [ ]:
def retreive():
  inp = input("Enter plant or disease: ")
  relations = rels[inp]
  if inp in diseases:
    print("disease")
  if inp in ingredients:
    print("plant")
  if inp in plants:
    print("medicinal plant")
  for (k,v) in relations:
    print(inp, v, k)

In [43]:
rels

{'பொடி': {('அதிமதுரம்', 'byprod'),
  ('அரை', 'byprod'),
  ('ஆகியவற்றைப்', 'byprod'),
  ('இப்பிலி', 'byprod'),
  ('இரண்டு', 'byprod'),
  ('இரண்டையும்', 'byprod'),
  ('இராம்', 'byprod'),
  ('இருவேளை', 'byprod'),
  ('இலவம்', 'byprod'),
  ('உப்பு', 'byprod'),
  ('ஊறவை', 'byprod'),
  ('எருக்கம்', 'byprod'),
  ('எலுமிச்சம்', 'byprod'),
  ('ஏலக்கா', 'byprod'),
  ('ஒன்று', 'byprod'),
  ('கசகசா', 'byprod'),
  ('கடுக்காய்', 'byprod'),
  ('கட்டை', 'byprod'),
  ('கம்', 'byprod'),
  ('கராம்', 'byprod'),
  ('கழற்சிக்காய்', 'byprod'),
  ('காயவை', 'byprod'),
  ('கால்', 'byprod'),
  ('கிராம்பு', 'byprod'),
  ('கிழங்கு', 'byprod'),
  ('கிழங்கை', 'byprod'),
  ('கீரையுடன்', 'byprod'),
  ('கீரையை', 'byprod'),
  ('கீழாநெல்லி', 'byprod'),
  ('குச்சி', 'byprod'),
  ('கொட்டை', 'byprod'),
  ('சடை', 'byprod'),
  ('சர்க்கீரை', 'byprod'),
  ('சிட்டிகை', 'byprod'),
  ('சீரகம்', 'byprod'),
  ('சுக்கு', 'byprod'),
  ('செய்துகொள்ளவ்', 'byprod'),
  ('ஜாதிக்காய்', 'byprod'),
  ('தனமும்', 'byprod'),
  ('தனியா', 'byprod')

In [ ]:
key = "start"
while(key != 'stop'):
  retreive()
  print()
  key = input("Enter key: ")
  print()

### CONVERT TO JSON FOR RETREIVAL

In [47]:
json_data = {}
def create_json():
  for key in rels.keys():
    if key in plants:
      json_data[key] = {
          "Category": 'medicinal plant',
          "Supporting Ingredients": set(),
          "Diseases Cured": set(),
          "By-Products": set(),
          "Forms of the plant": set(),
          "Similar Plants": set()
      }
    elif key in ingredients:
      json_data[key] = {
          "Category": 'ingredient/plant',
          "Supporting Ingredients": set(),
          "Diseases Cured": set(),
          "By-Products": set(),
          "Forms of the plant": set(),
          "Similar Plants": set()
      }
    elif key in diseases:
      json_data[key] = {
          "Category": 'disease/condition',
          "Similar Diseases": set(),
          "Cure: Medicinal Plants": set(),
          "Cure: Other plants/ingredients": set(),
      }
  return json_data

In [48]:
def update_json():
  json_data = create_json()
  for key in rels.keys():
    if key in byProducts:
      for (k,v) in rels[key]:
        if k in json_data.keys() and v == "byprod":
          json_data[k]["By-Products"].add(key)
        else:
          if k in plants:
            json_data[k] = {
              "Category": 'medicinal plant',
              "Supporting Ingredients": set(),
              "Diseases Cured": set(),
              "By-Products": {key},
              "Forms of the plant": set(),
              "Similar Plants": set()
          }
          if k in ingredients:
            json_data[k] = {
                "Category": 'ingredient/plant',
                "Supporting Ingredients": set(),
                "Diseases Cured": set(),
                "By-Products": {key},
                "Forms of the plant": set(),
                "Similar Plants": set()
            }
    else:
      for (k,v) in rels[key]:
        if json_data[key]['Category'] == 'medicinal plant':
            if v == 'supp-ing':
              json_data[key]["Supporting Ingredients"].add(k)
            elif v == 'dis':
              json_data[key]["Diseases Cured"].add(k)
            elif v == 'byprod':
              json_data[key]["By-Products"].add(k)
            elif v == 'sim-plant':
              json_data[key]["Forms of the plant"].add(k)
            json_data[key]["Similar Plants"].update(get_semantically_similar_plants(rels, key))

        if json_data[key]["Category"] == 'ingredient/plant':
            if v == 'supp-ing':
              json_data[key]["Supporting Ingredients"].add(k)
            elif v == 'dis':
              json_data[key]["Diseases Cured"].add(k)
            elif v == 'byprod':
              json_data[key]["By-Products"].add(k)
            elif v == 'sim-plant':
              json_data[key]["Forms of the plant"].add(k)
            json_data[key]["Similar Plants"].update(get_semantically_similar_plants(rels, key))
        
        if json_data[key]['Category'] == 'disease/condition':
            if v == 'sim-dis':
              json_data[key]["Similar Diseases"].add(k)
            elif v == 'med-plant':
              json_data[key]["Cure: Medicinal Plants"].add(k)
            elif v == 'ing':
              json_data[key]["Cure: Other plants/ingredients"].add(k)
  return json_data

In [49]:
import json

json_data = update_json()
for key in json_data:
  for sec_key in json_data[key]:
    if(isinstance(json_data[key][sec_key], set) and len(json_data[key][sec_key])>0):
      json_data[key][sec_key] = list(json_data[key][sec_key])
    if(len(json_data[key][sec_key])==0):
      json_data[key][sec_key] = '-'
with open('/content/drive/MyDrive/Final Year Project/Outputs/relationships.json', "w") as outfile:
  outfile.write(json.dumps(json_data, ensure_ascii=False))